# Install and Import Requirements

In [13]:
# Credit to: https://jupyter.brynmawr.edu/services/public/dblank/CS371%20Cognitive%20Science/2016-Fall/Programming%20a%20Chess%20Player.ipynb for parts of function to play a game and parts of human player input class
# and https://medium.com/dscvitpune/lets-create-a-chess-ai-8542a12afef for parts of piece value table calculation

!pip install python-chess
import chess
import random
from IPython.display import display, HTML, clear_output
import time

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Define helper functions and random/human players (functions to choose from legal moves given a board position)

In [14]:
#Helper functions
def which_color(player):
    return "White" if player == chess.WHITE else "Black"

def input_move(board):
  move = None
  algebraic = input("Input move [q to quit]> ")
  if algebraic and algebraic[0] == "q":
    raise KeyboardInterrupt()
  try:
    move = board.parse_san(algebraic)
  except:
    move = None
  return move

In [15]:
def random_player(board):
    move = random.choice(list(board.legal_moves)) 
    return move

def human_player(board):
  move = input_move(board)
  legal_moves = [move for move in board.legal_moves]
  while move not in legal_moves:
    print("Invalid")
    move = input_move(board)
  return move

# Main function to take players and play a game

In [16]:
def play_game(player1, player2, pause=0.1):
    board = chess.Board()
    try:
      while not board.is_game_over(claim_draw=True):
        #Get Move from player whose turn it is
        if board.turn == chess.WHITE:
          move = player1(board)
        else:
          move = player2(board)
        move_notation = board.san(move) #Convert to standard readable notation
        board.push(move) #Make move


        # Display Board and Info
        move_number = (int)((len(board.move_stack)+2)/2)
        html = "<b>Move %s %s, Last Move: '%s':</b><br/>%s" % (
                    move_number, which_color(board.turn), move_notation, board._repr_svg_())
        clear_output(wait=True)
        display(HTML(html))    
        time.sleep(pause)
    except KeyboardInterrupt:
      msg = "Keyboard Interrupt"
      return (None, msg, board)

    #When Game ends, display result:
    result = None
    if board.is_checkmate():
      msg = "Checkmate! " + which_color(not board.turn) + " wins."
      result = not board.turn
    elif board.is_stalemate():
      msg = "Draw: stalemate"
    elif board.is_fivefold_repetition():
      msg = "Draw: 5-fold repetition"
    elif board.is_insufficient_material():
      msg = "Draw: insufficient material"
    elif board.can_claim_draw():
      msg = "Draw: claim"
    print(msg)

    move_sequence = []
    for j in range(len(board.move_stack)):
      move = board.pop()
      move_sequence.append(board.san(move))

    print("Moves: ")
    i = 1
    white = True
    for k in range(len(move_sequence)-1, -1, -1):
      if white:
        print(str(i) + ". " + move_sequence[k])
        white = False
      else:
        print(str(i) + "... " + move_sequence[k])
        white = True
        i+=1
      
    return (result, msg, board)

# Write AI player

## BetaZero

In [17]:
def betazero_player(board, depth=2):

  if(board.turn):
    best_eval = float('-inf')
    best_move = None
    for move in board.legal_moves:
      board.push(move)
      eval = min_search(board, float('-inf'), float('inf'), depth)
      board.pop()
      if eval > best_eval:
        best_eval = eval
        best_move = move
    return best_move

  else:
    best_eval = float('inf')
    best_move = None
    for move in board.legal_moves:
      board.push(move)
      eval = max_search(board, float('-inf'), float('inf'), depth)
      board.pop()
      if eval < best_eval:
        best_eval = eval
        best_move = move
    return best_move

def omegazero_player(board, depth=1):

  if(board.turn):
    best_eval = float('-inf')
    best_move = None
    for move in board.legal_moves:
      board.push(move)
      eval = min_search(board, float('-inf'), float('inf'), depth)
      board.pop()
      if eval > best_eval:
        best_eval = eval
        best_move = move
    return best_move

  else:
    best_eval = float('inf')
    best_move = None
    for move in board.legal_moves:
      board.push(move)
      eval = max_search(board, float('-inf'), float('inf'), depth)
      board.pop()
      if eval < best_eval:
        best_eval = eval
        best_move = move
    return best_move

## Min-max search with alpha-beta pruning

In [18]:
def max_search(board, alpha, beta, depth):
  if board.is_checkmate():
    if board.turn:
      return -9999
    else:
      return 9999
  if board.is_stalemate():
    return 0
  if board.is_insufficient_material():
    return 0
  if board.can_claim_draw():
    return 0

  if depth == 0:
    return position_evaluation_function(board)

  best_eval = float('-inf')

  for move in board.legal_moves:
    board.push(move)
    eval = min_search(board, alpha, beta, depth-1)
    board.pop()
    if eval > best_eval:
      best_eval = eval
      alpha = eval
    if eval > beta:
      return eval

  return best_eval

def min_search(board, alpha, beta, depth):
  if board.is_checkmate():
    if board.turn:
      return -9999
    else:
      return 9999
  if board.is_stalemate():
      return 0
  if board.is_insufficient_material():
      return 0
  if board.can_claim_draw():
    return 0

  if depth == 0:
    return position_evaluation_function(board)

  best_eval = float('inf')

  for move in board.legal_moves:
    board.push(move)
    eval = max_search(board, alpha, beta, depth-1)
    board.pop()

    if eval < best_eval:
      best_eval = eval
      beta = eval
    if eval < alpha:
      return eval

  return best_eval


## Position Evaluation Function

### Position Table

In [19]:
pawntable = [
    0, 0, 0, 0, 0, 0, 0, 0,
    5, 10, 10, -20, -20, 10, 10, 5,
    5, 3, -15, 0, 0, -5, 4, 5,
    0, 0, 0, 20, 20, 0, 0, 0,
    5, 5, 10, 25, 25, 10, 5, 5,
    10, 10, 20, 30, 30, 20, 10, 10,
    50, 50, 50, 50, 50, 50, 50, 50,
    0, 0, 0, 0, 0, 0, 0, 0]

knightstable = [
    -50, -40, -30, -30, -30, -30, -40, -50,
    -40, -20, 0, 5, 5, 0, -20, -40,
    -30, 5, 10, 15, 15, 10, 5, -30,
    -30, 0, 15, 20, 20, 15, 0, -30,
    -30, 5, 15, 20, 20, 15, 5, -30,
    -30, 0, 10, 15, 15, 10, 0, -30,
    -40, -20, 0, 0, 0, 0, -20, -40,
    -50, -40, -30, -30, -30, -30, -40, -50]

bishopstable = [
    -20, -10, -10, -10, -10, -10, -10, -20,
    -10, 12, 0, 0, 0, 0, 12, -10,
    -10, 10, 10, 10, 10, 10, 10, -10,
    -10, 0, 10, 10, 10, 10, 0, -10,
    -10, 5, 5, 10, 10, 5, 5, -10,
    -10, 0, 5, 10, 10, 5, 0, -10,
    -10, 0, 0, 0, 0, 0, 0, -10,
    -20, -10, -10, -10, -10, -10, -10, -20]

rookstable = [
    0, 0, 0, 5, 5, 0, 0, 0,
    -5, 0, 0, 0, 0, 0, 0, -5,
    -5, 0, 0, 0, 0, 0, 0, -5,
    -5, 0, 0, 0, 0, 0, 0, -5,
    -5, 0, 0, 0, 0, 0, 0, -5,
    -5, 0, 0, 0, 0, 0, 0, -5,
    5, 10, 10, 10, 10, 10, 10, 5,
    0, 0, 0, 0, 0, 0, 0, 0]

queenstable = [
    -20, -10, -10, -5, -5, -10, -10, -20,
    -10, 0, 0, 0, 0, 0, 0, -10,
    -10, 5, 5, 5, 5, 5, 0, -10,
    0, 0, 5, 5, 5, 5, 0, -5,
    -5, 0, 5, 5, 5, 5, 0, -5,
    -10, 0, 5, 5, 5, 5, 0, -10,
    -10, 0, 0, 0, 0, 0, 0, -10,
    -20, -10, -10, -5, -5, -10, -10, -20]

kingstable = [
    20, 30, 10, 0, 0, 10, 30, 20,
    20, 20, 0, 0, 0, 0, 20, 20,
    -10, -20, -20, -20, -20, -20, -20, -10,
    -20, -30, -30, -40, -40, -30, -30, -20,
    -30, -40, -40, -50, -50, -40, -40, -30,
    -30, -40, -40, -50, -50, -40, -40, -30,
    -30, -40, -40, -50, -50, -40, -40, -30,
    -30, -40, -40, -50, -50, -40, -40, -30]

def get_position_value(board):
  pawnsq = sum([pawntable[i] for i in board.pieces(chess.PAWN, chess.WHITE)])
  pawnsq = pawnsq + sum([-pawntable[chess.square_mirror(i)]
                        for i in board.pieces(chess.PAWN, chess.BLACK)])
  
  knightsq = sum([knightstable[i] for i in board.pieces(chess.KNIGHT, chess.WHITE)])
  knightsq = knightsq + sum([-knightstable[chess.square_mirror(i)]
                            for i in board.pieces(chess.KNIGHT, chess.BLACK)])
  
  bishopsq = sum([bishopstable[i] for i in board.pieces(chess.BISHOP, chess.WHITE)])
  bishopsq = bishopsq + sum([-bishopstable[chess.square_mirror(i)]
                            for i in board.pieces(chess.BISHOP, chess.BLACK)])
  
  rooksq = sum([rookstable[i] for i in board.pieces(chess.ROOK, chess.WHITE)])
  rooksq = rooksq + sum([-rookstable[chess.square_mirror(i)]
                        for i in board.pieces(chess.ROOK, chess.BLACK)])
  
  queensq = sum([queenstable[i] for i in board.pieces(chess.QUEEN, chess.WHITE)])
  queensq = queensq + sum([-queenstable[chess.square_mirror(i)]
                          for i in board.pieces(chess.QUEEN, chess.BLACK)])
  
  kingsq = sum([kingstable[i] for i in board.pieces(chess.KING, chess.WHITE)])
  kingsq = kingsq + sum([-kingstable[chess.square_mirror(i)]
                        for i in board.pieces(chess.KING, chess.BLACK)])
  
  return pawnsq + knightsq + bishopsq + rooksq + queensq + kingsq

### Material Count

In [20]:
def get_material_count(board):
  wp = len(board.pieces(chess.PAWN, chess.WHITE))
  bp = len(board.pieces(chess.PAWN, chess.BLACK))
  wn = len(board.pieces(chess.KNIGHT, chess.WHITE))
  bn = len(board.pieces(chess.KNIGHT, chess.BLACK))
  wb = len(board.pieces(chess.BISHOP, chess.WHITE))
  bb = len(board.pieces(chess.BISHOP, chess.BLACK))
  wr = len(board.pieces(chess.ROOK, chess.WHITE))
  br = len(board.pieces(chess.ROOK, chess.BLACK))
  wq = len(board.pieces(chess.QUEEN, chess.WHITE))
  bq = len(board.pieces(chess.QUEEN, chess.BLACK))

  return 100 * (wp - bp) + 300 * (wn - bn) + 310 * (wb - bb) + 500 * (wr - br) + 900 * (wq - bq)

### Squares controlled

In [21]:
def get_square_control(board):
  sum = 0
  for i in range(64):
    attacks = len(list(board.attacks(i)))
    if attacks == 0:
      continue
    piece = board.piece_at(i)
    if piece.color:
      sum += attacks
    else:
      sum -= attacks

  return sum * 20

  # sum = 0
  # for i in range(64):
  #   attacks = len(list(board.attacks(i)))
  #   piece = board.piece_at(i)
  #   if piece is not None:
  #     if piece.color:
  #       sum += attacks
  #     else:
  #       sum -= attacks

  # return sum * 5

### Combine evaluations

In [22]:
materialisticness = 1.0
positionality = 1.5
control = 0.8

def position_evaluation_function(board):
  material = get_material_count(board)
  piece_positioning = get_position_value(board)
  square_control = get_square_control(board)

  return (material * materialisticness) + (piece_positioning * positionality) + (square_control * control)

# Play against AI

In [23]:
play_game(betazero_player, human_player)
print("Game Over")

Game Over


# A couple of saved games

Moves: 
1. Nf3
1... Nf6
2. Nc3
2... Nc6
3. e3
3... Ng4
4. e4
4... e5
5. Bc4
5... Qf6
6. Bxf7+
6... Qxf7
7. d4
7... Nxd4
8. O-O
8... Qh5
9. Nd5
9... Nxf3+
10. Kh1
10... Qxh2#

Checkmate! White wins.
Moves: 
1. d4
1... e6
2. Qd3
2... Bb4+
3. Bd2
3... Qe7
4. a3
4... Bd6
5. Nc3
5... Qf6
6. Ne4
6... Qf5
7. Nxd6+
7... cxd6
8. Qxf5
8... exf5
9. Nf3
9... Nf6
10. Bf4
10... Nc6
11. Bxd6
11... Ne4
12. Bf4
12... g5
13. Nxg5
13... Nxg5
14. d5
14... Ne7
15. Bxg5
15... Nxd5
16. O-O-O
16... h6
17. Bxh6
17... Nc3
18. bxc3
18... Rxh6
19. e3
19... Ke7
20. Bc4
20... b5
21. Bxb5
21... Rb8
22. Rd5
22... d6
23. Bc6
23... Be6
24. Ra5
24... a6
25. Rxa6
25... Rhh8
26. Ra7+
26... Kf6
27. Kd2
27... Kg7
28. Bf3
28... d5
29. h4
29... Rhc8
30. h5
30... Rh8
31. Be2
31... Rh6
32. Rh4
32... Rhh8
33. h6+
33... Rxh6
34. Rxh6
34... Kxh6
35. Re7
35... Kg7
36. Ra7
36... Rb1
37. Re7
37... Rb8
38. Rc7
38... Rb1
39. Ra7
39... Rb8
40. Re7
40... Rb1
41. Rc7
41... Rb8
42. Kc1
42... Rb6
43. Kd1
43... Rb8
44. Ke1
44... Rh8
45. Rb7
45... Rh2
46. g3
46... Rh3
47. Kd2
47... Rh8
48. Bf3
48... Rc8
49. Rb4
49... Rh8
50. Rb1
50... Rc8
51. Rb7
51... Rc4
52. Rb4
52... Rc8
53. Rh4
53... Rb8
54. Rb4
54... Rh8
55. Rb6
55... Rc8
56. Rb5
56... Rc4
57. Bxd5
57... Bxd5
58. Rxd5
58... Kg6
59. Rb5
59... Rc6
60. Kd3
60... Re6
61. Ke2
61... Kf6
62. Rd5
62... Kg6
63. a4
63... Kf6
64. a5
64... Kg6
65. Rb5
65... Kf6
66. Rb6
66... Kg7
67. Rxe6
67... fxe6
68. a6
68... e5
69. a7
69... f4
70. a8=Q
70... fxe3
71. fxe3
71... Kg6
72. Qe8+
72... Kf6
73. e4
73... Kg7
74. Qxe5+
74... Kg8
75. Qf6
75... Kh7
76. c4
76... Kg8
77. Kf3
77... Kh7
78. Kg2
78... Kg8
79. c5
79... Kh7
80. c6
80... Kg8
81. c7
81... Kh7
82. Qf7+
82... Kh6
83. c8=Q
83... Kg5
84. Qh8
84... Kg4
85. Qhg8#
Game Over

Checkmate! Black wins.
Moves: 
1. g3
1... d5
2. Nh3
2... e5
3. f3
3... Bc5
4. Ng5
4... Qxg5
5. c4
5... Nf6
6. e3
6... Nc6
7. Nc3
7... Bb4
8. Ne2
8... Qxe3
9. Bh3
9... Bxh3
10. g4
10... Bg2
11. g5
11... Bxh1
12. h4
12... Bxf3
13. h5
13... Ng4
14. a3
14... Rg8
15. Rb1
15... Rh8
16. Kf1
16... Bxd2
17. b3
17... Rg8
18. Qxd2
18... Qf2#
Game Over

Checkmate! White wins.
Moves: 
1. d4
1... Nc6
2. d5
2... Nd4
3. Qxd4
3... c6
4. Nc3
4... h5
5. Nf3
5... e6
6. Bg5
6... Bb4
7. Bxd8
7... Rh6
8. Qxb4
8... d6
9. Bg5
9... Rh7
10. Qxd6
10... Bd7
11. dxe6
11... b6
12. Qxd7+
12... Kf8
13. Qxf7#
Game Over